In [221]:
import lxml.html as LH
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import string
import warnings
import os
import urllib.request
import datetime
from itertools import compress
warnings.filterwarnings('ignore')

In [ ]:
os.chdir('C:/Users/mnest/Google Drive/PYWD')

In [28]:
datetime.datetime.now()

datetime.datetime(2017, 10, 5, 10, 11, 15, 635636)

In [210]:
 class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = bs(response.text, 'lxml')
            return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th')
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags[1:]:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [330]:
class bbref_scrape:
    def get_player_ids(url):
        r = requests.get(url)
        all_tags = bs(r.content, "html.parser")
        tmp = [x for x in all_tags.find_all("td", class_ = "left")]
        ids = []
        for x in tmp:
            try:
                ids.append(x["data-append-csv"])
            except:
                next
        return(ids)

    def get_player_links(url):
        r = requests.get(url)
        all_tags = bs(r.content, "html.parser")
        ids_bool = [bool(re.search(pattern="players/\w/.+", string = x["href"])) for x in all_tags.find_all("a")]
        ids = list(compress([x["href"] for x in all_tags.find_all("a")], ids_bool))
        return([re.sub(pattern="[.](html)",string=x, repl="") for x in ids])
    
    def get_player_gamelogs(sport_type, link, year):
        if(sport_type == "basketball"):
            ref_link = "basketball-reference"
        else:
            ref_link = "hockey-reference"

        bbrefID = re.findall(string=link, pattern="(?<=[/])\w+|\d+")[2]
        url = "https://www."+ ref_link + link + "/gamelog/" + str(year)
        r = requests.get(url)
        all_tags = LH.fromstring(r.content)
    
        for table in all_tags.xpath('//*[@id="pgl_basic"]'):
            header = [text(th) for th in table.xpath('//th')][1:30]
            data = [[text(td) for td in tr.xpath('td')]  
                    for tr in table.xpath('//tr')][1:]
            data = [row for row in data if len(row)==len(header)]
            data = pd.DataFrame(data, columns = header)
            df = pd.concat([pd.DataFrame({"bbrefID":[bbrefID for bbref in range(len(data))]}), data], axis=1)
            return(df)

In [331]:
hp = HTMLTableParser()
#NBA
player_pg = "https://www.basketball-reference.com/leagues/NBA_2017_per_game.html"
player_p100 = "https://www.basketball-reference.com/leagues/NBA_2017_per_poss.html"
player_p36 = "https://www.basketball-reference.com/leagues/NBA_2017_per_minute.html"
player_advanced = "https://www.basketball-reference.com/leagues/NBA_2017_advanced.html"
#NHL
skater_basic = "https://www.hockey-reference.com/leagues/NHL_2018_skaters.html"
skater_advanced = "https://www.hockey-reference.com/leagues/NHL_2018_skaters-advanced.html"

In [196]:
#NBA Game Logs
ids = bbref_scrape.get_player_links(player_pg)
game_logs_df = pd.concat([bbref_scrape.get_player_gamelogs(sport_type = "basketball", x, 2017) for x in ids])
game_logs_df.to_csv("C:\\Users\\neste\\Google Drive\\PYWD\\NBA\\game_logs\\game_logs.csv")

In [333]:
#NHL Game Logs
ids = bbref_scrape.get_player_links(skater_basic)
game_logs_df = pd.concat([bbref_scrape.get_player_gamelogs(sport_type = "hockey", x, 2018) for x in ids])
game_logs_df.to_csv("C:\\Users\\neste\\Google Drive\\PYWD\\NHL\\game_logs\\game_logs.csv")

['/players/m/mcdavco01',
 '/players/s/simmowa01',
 '/players/m/matthau01',
 '/players/v/vanrija01',
 '/players/g/gostish01',
 '/players/v/voracja01',
 '/players/l/labanke01',
 '/players/m/marlepa01',
 '/players/p/pietral01',
 '/players/c/crosbsi01',
 '/players/k/kadrina01',
 '/players/m/marnemi01',
 '/players/s/schenbr01',
 '/players/s/stastpa01',
 '/players/b/bozakty01',
 '/players/d/draisle01',
 '/players/g/guentja01',
 '/players/m/malkiev01',
 '/players/r/rustbr01',
 '/players/s/schwaja01',
 '/players/s/sobotvl01',
 '/players/c/coutulo01',
 '/players/g/giroucl01',
 '/players/m/maattol01',
 '/players/n/nylanwi01',
 '/players/p/parayco01',
 '/players/p/perrema01',
 '/players/s/scheima01',
 '/players/s/schulju01',
 '/players/s/shearco01',
 '/players/w/wealjo01',
 '/players/b/borgman01',
 '/players/b/burnsbr01',
 '/players/b/byfugdu01',
 '/players/c/carrico01',
 '/players/c/coutuse01',
 '/players/d/dillobr01',
 '/players/g/gardija01',
 '/players/g/gunnaca01',
 '/players/k/komarle01',
 '